In [ ]:
import pandas as pd

def calculate_and_compare_pnl(csv_file1, csv_file2):
    # Read the CSV files
    df1 = pd.read_csv(csv_file1)
    df2 = pd.read_csv(csv_file2)

    # Aggregate data in the first CSV
    aggregation_functions = {'Size': 'sum', 'MWH': 'sum', 'PNL': 'sum'}
    df1_aggregated = df1.groupby(['Trades', 'Region', 'Location', 'Option Type', 'Trade Price', 'Mark Price'], as_index=False).aggregate(aggregation_functions)

    # Prepare the second CSV for PNL calculation
    df2 = df2.merge(df1_aggregated[['Region', 'Location', 'Trade Price', 'Mark Price', 'Option Type']], on=['Region', 'Location', 'Trade Price'], how='left')

    # Define a function for PNL calculation based on Option Type
    def calculate_pnl(row):
        if row['Option Type'] == 'power':
            return row['Size'] * (row['Mark Price'] - row['Trade Price'])
        elif row['Option Type'] == 'gas':
            return row['Size'] * (row['Mark Price'] - row['Trade Price']) * 100
        else:
            return None  # or some default calculation

    # Apply the PNL calculation
    df2['Calculated PNL'] = df2.apply(calculate_pnl, axis=1)

    # Merge with the first CSV to compare PNL
    comparison = df2.merge(df1_aggregated[['Trades', 'PNL']], left_on='Trade', right_on='Trades', how='left')

    # Highlight differences in PNL
    comparison['PNL Difference'] = comparison['Calculated PNL'] - comparison['PNL']
    comparison['Highlight'] = comparison['PNL Difference'].apply(lambda x: 'RED' if x != 0 else '')

    return comparison[['Trade', 'PNL', 'Calculated PNL', 'Highlight']]

# Usage Example
# Replace 'csv1.csv' and 'csv2.csv' with your actual file paths
result = calculate_and_compare_pnl('csv1.csv', 'csv2.csv')
print(result)
